In [1]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-common_gen")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-common_gen")



/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.7/site-packages/transformers/models/auto/modeling_auto.py:1116: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [8]:


prompt = "team run drill field"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# generate up to 30 tokens
outputs = model.generate(input_ids, do_sample=True, max_length=20, top_p=0.95,num_return_sequences=10,num_beams=1)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['a teddy bear runs over as the boys team go into a field of',
 'The football team is running drills on the field.',
 'A man runs the drill in an open field as his team prepares for a match.',
 'The men are running along an artificial grass field on their training team.',
 'A group of team members are running drills through the field.',
 'a team is running drill in a field',
 'A soccer team runs down a field during drills.',
 'A team runs through a field of clay during drills.',
 'A team runs on a field during drills.',
 'A team of people are running drills in a field.']

In [21]:

def gen_sentence(words, max_length=20, beam_size= 10):
    input_text = words
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length, num_beams=beam_size, do_sample=True, top_p=0.9, num_return_sequences=beam_size)

    return [tokenizer.decode(output[i], skip_special_tokens=True) for i in range(len(output))]


# output: digging a hole in the ground to plant trees

words = "tree plant ground hole dig"
words = "ride wear hat horse"
words = " ".join(['balloon', 'couch', 'blow', 'sit'])
# words =' '.join(['snow', 'stand', 'dog', 'look']) 
# words = ' '.join(['front', 'trick', 'perform', 'crowd'])
gen_sentence(words)

['A man sitting on a couch blowing balloons.',
 'A man sitting on a couch blowing balloons.',
 'A man sitting on a couch blowing balloons.',
 'A man sitting on a couch blowing balloons.',
 'A man sitting on a couch blowing balloons.',
 'A man sitting on a couch blowing balloons.',
 'A man sitting on a couch blowing balloons.',
 'A man sitting on a couch blowing balloons.',
 'A man sitting on a couch blowing balloons.',
 'A man sitting on a couch blowing balloons.']

In [19]:

def gen_sentence(words, max_length=20, beam_size= 10):
    input_text = words
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length, num_beams=beam_size, num_beam_groups=5,diversity_penalty=0.5, num_return_sequences=beam_size)

    return [tokenizer.decode(output[i], skip_special_tokens=True) for i in range(len(output))]


# output: digging a hole in the ground to plant trees

words = "tree plant ground hole dig"
words = "ride wear hat horse"
# words =' '.join(['snow', 'stand', 'dog', 'look']) 
# words = ' '.join(['front', 'trick', 'perform', 'crowd'])
gen_sentence(words)

/export/home/anaconda3/lib/python3.9/site-packages/transformers/generation_beam_search.py:196: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect.`max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`,or `group_beam_search(...)`.
  warnings.warn(
/export/home/anaconda3/lib/python3.9/site-packages/transformers/generation_utils.py:2451: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size
/export/home/anaconda3/lib/python3.9/site-packages/transformers/generation_utils.py:2474: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of 

['A man wearing a hat rides a horse.',
 'A man riding a horse in a hat.',
 'A man wearing a hat rides a horse..',
 'A young girl wearing a hat rides a horse.',
 'a man wears a hat while riding his horse',
 'A man wearing a hat rides a horse.',
 'A man riding a horse wearing a hat.',
 'A man riding a horse in a hat..',
 'A man is riding a horse wearing a hat.',
 'A woman is riding a horse wearing a hat.']

In [3]:
from collections import defaultdict
def distinctness(outputs, EOT_TOKEN='.'):
    cw = "default"
    d1, d2, d3 = defaultdict(lambda: set()), defaultdict(lambda: set()), defaultdict(lambda: set())
    total_words = defaultdict(lambda: 0)

    for o in outputs:
        o = o.replace(EOT_TOKEN, ' ').strip().split(' ')
        o = [str(x) for x in o]
        total_words[cw] += len(o)
        d1[cw].update(o)
        for i in range(len(o) - 1):
            d2[cw].add(o[i] + ' ' + o[i+1])
        for i in range(len(o) - 2):
            d3[cw].add(o[i] + ' ' + o[i+1] + ' ' + o[i+2])
    return_info = []
    avg_d1, avg_d2, avg_d3 = 0, 0, 0
    for cw in total_words.keys():
        return_info.append((cw, 'DISTINCTNESS', len(d1[cw]) / total_words[cw], len(d2[cw]) / total_words[cw], len(d3[cw]) / total_words[cw]))
        avg_d1 += len(d1[cw]) / total_words[cw]
        avg_d2 += len(d2[cw]) / total_words[cw]
        avg_d3 += len(d3[cw]) / total_words[cw]
    avg_d1, avg_d2, avg_d3 = avg_d1 / len(total_words.keys()), avg_d2 / len(total_words.keys()), avg_d3 / len(total_words.keys())
    return return_info, (avg_d1, avg_d2, avg_d3)

In [6]:
distinctness(["this is a sentence","this is another","this is another","this is another","asdas asd sd s ds ds dasd asd asd as "])

([('default',
   'DISTINCTNESS',
   0.5217391304347826,
   0.5652173913043478,
   0.4782608695652174)],
 (0.5217391304347826, 0.5652173913043478, 0.4782608695652174))

In [1]:
from datasets import load_dataset
answersumm = load_dataset("alexfabbri/answersumm")


Using custom data configuration alexfabbri--answersumm-fc26db571c9b9cb8


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/alexfabbri___json/alexfabbri--answersumm-fc26db571c9b9cb8/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
dev = answersumm['validation']
print(len(dev))
print(dev[3])

500
{'answers': [{'sents': [{'text': "As a hiring manager I've asked questions like this in the past and from my perspective", 'label': [0], 'label_summ': [0], 'cluster_id': [[-1]]}, {'text': 'I want to hear a story where you had to use your brain to decide on a course of action and how you implemented it.', 'label': [0], 'label_summ': [0], 'cluster_id': [[-1]]}, {'text': 'Was the decision technically sound?', 'label': [0], 'label_summ': [0], 'cluster_id': [[-1]]}, {'text': 'Did you know the path to success, and were you confident?', 'label': [0], 'label_summ': [0], 'cluster_id': [[-1]]}, {'text': "Were there inter-personal, inter-department, or other 'fuzzy' issues that could have prevented it from being successful?", 'label': [0], 'label_summ': [0], 'cluster_id': [[-1]]}, {'text': 'Can you show me some passion that you were responsible for it?', 'label': [0], 'label_summ': [0], 'cluster_id': [[-1]]}, {'text': "If you can't think of anything technical that closely relates to the job d

In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("Unbabel/gec-t5_small")
tokenizer = T5Tokenizer.from_pretrained('t5-small')


Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

/export/home/anaconda3/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [8]:

sentence = "I like to swimming and walking to the classrom. Hoowever, today is a different day for me."
sentence = "Also the work of the media , television , magazines , and every stuff made for publicity affects the emotions of many people around the world , encouraging them to buy and buy anything they can ."
tokenized_sentence = tokenizer('gec: ' + sentence, max_length=128, truncation=True, padding='max_length', return_tensors='pt')
corrected_sentence = tokenizer.decode(
    model.generate(
        input_ids = tokenized_sentence.input_ids,
        attention_mask = tokenized_sentence.attention_mask, 
        max_length=128,
        num_beams=5,
        early_stopping=True,
    )[0],
    skip_special_tokens=True, 
    clean_up_tokenization_spaces=True
)
print(corrected_sentence) # -> I like swimming.


I like swimming and walking to the classroom. However, today is a different day for me.


In [1]:
import statistics
import random
import torch
import datasets

from collections import defaultdict

from datasets import load_metric

from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification

from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

mauve = load_metric('mauve')
rouge = load_metric('rouge')
meteor = load_metric( 'meteor')
bleu = load_metric('bleu')

/tmp/ipykernel_79625/2172818720.py:17: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  mauve = load_metric('mauve')
[nltk_data] Downloading package wordnet to /export/home/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /export/home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /export/home/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
predictions = ["It is a guide to action which ensures that the military always obeys the commands of the party"]
references = ["It is a guide to action that ensures that the military will forever heed Party commands","It is a guide to action that ensures that the military will forever heed Party commands"]
results = meteor.compute(predictions=predictions, references=references)
print(results)

ValueError: Mismatch in the number of predictions (1) and references (2)

In [17]:
import spacy
nlp = spacy.load("en_core_web_sm")


from spacy.pipeline.tagger import DEFAULT_TAGGER_MODEL
config = {"model": DEFAULT_TAGGER_MODEL}


def tokenize(sentences):
    new_sentence_list = []
    for sent in sentences:
        a = ''
        for token in nlp(sent):
            print(token.text)
            a += token.text
            a += ' '

        new_sentence_list.append(a.rstrip())
    return new_sentence_list



In [18]:
out = tokenize(['This is a   sentence? ','这是一句中文。'])
print(out)

This
is
a
  
sentence
?
这是一句中文
。
['This is a    sentence ?', '这是一句中文 。']


In [21]:
sentences = ['Im vergangenen Jahr verschärfte sich die Lage weiter, als das weltgrößte Erzeugerland China angesichts der schwächelnden heimischen Nachfrage massenhaft Stahl auf den Weltmarkt warf.']
nlp = spacy.load("de_core_news_sm")
sentences = ['this is an English sentence.']
nlp = spacy.load("en_core_web_sm")

doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

this is an English sentence.
this PRON nsubj
is AUX ROOT
an DET det
English ADJ amod
sentence NOUN attr
. PUNCT punct
